In [6]:
import pandas as pd
import json
from datetime import datetime

### `Abrimos el archivo "output_steam_games.json"`

In [7]:
archivo_json = "output_steam_games.json"
df_steam_games = pd.read_json(archivo_json, lines=True)

In [8]:
# Observamos el tipo de dato y si es necesario lo modificamos
df_steam_games.dtypes

publisher        object
genres           object
app_name         object
title            object
url              object
release_date     object
tags             object
reviews_url      object
specs            object
price            object
early_access    float64
id              float64
developer        object
dtype: object

Modificamos la columna "Release_date"

In [9]:
# Pasamos a string la columna "release_date"
df_steam_games['release_date'] = df_steam_games['release_date'].astype(str)

In [10]:
# Convertir la columna "release_date" a tipo de dato "datetime", los valores que no pueden pasarse a datetime los deja como "NaT"
df_steam_games['release_date']= pd.to_datetime(df_steam_games['release_date'], format='%Y-%m-%d', errors='coerce')

In [11]:
# Dejamos unicamente el año en "release_date"
df_steam_games['release_date'] = df_steam_games['release_date'].dt.year

In [12]:
# Borramos los nulos de la columna "release_date"
df_steam_games = df_steam_games.dropna(subset= "release_date")

Modificamos la columna "Price"

In [13]:
# La columna tiene datos en numeros y string
df_steam_games['price'].value_counts()
# Pasamos a tipo numerico y los que no puede pasarlo a numerico los pone como nulos
df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')
# Los nulos los reemplaza por "Free"
df_steam_games['price'].fillna("Free", inplace=True)

In [14]:
df_steam_games.to_csv("df_steam_games.csv",index=False)

### ``Abrimos el archivo "australian_user_reviews.json"``

In [15]:
# Abrimos el archivo "australian_user_reviews.json"
with open("australian_user_reviews.json","r",encoding="utf-8") as f:
    data = f.readlines() # readlines() crea en cada linea una lista de cadenas

In [16]:
# Hacemos un dataframe df_users
df_users = [eval(line.strip()) for line in data]
df_users = pd.DataFrame(df_users)

In [17]:
# Desanidamos las columnas reviews

data_desanidada_users = []
for indice, fila in df_users.iterrows():
    reviews = fila["reviews"]

    # Verificar si items no es None y es iterable
    if reviews is not None and isinstance(reviews, (list, tuple)):
        for i in reviews:
          Fila_Nueva = {
          'funny': i.get("funny"," "),
          'posted': i.get("posted"," "),
          'last_edited': i.get("last_edited"," "),
          'item_id': i.get("item_id"," "),
          'helpful':   i.get("helpful"," "),
          'recommend': i.get("recommend"," "),
          'review':    i.get("review"," ")

            }
          data_desanidada_users.append(Fila_Nueva)
    else:
        pass

In [18]:
# Hacemos un dataframe con la lista data_desanidada_users
data_desanidada_users = pd.DataFrame(data_desanidada_users)

In [19]:
# Concatenamos df_steam_games y data_desanidada_users
df_users = pd.concat([df_users, data_desanidada_users], axis=1)

In [20]:
df_users = df_users.dropna(subset= "item_id")

In [21]:
df_users = df_users.dropna(subset= "user_id")

In [22]:
# Una vez transformados los datos y columnas hacemos una exportacion en archivo csv
df_users.to_csv("df_users.csv",index=False)

### ``Abrimos el archivo "australian_users_items.json"``

In [23]:
# Abrimos el archivo "australian_users_items.json"
with open("australian_users_items.json","r",encoding="utf-8") as f:
    data = f.readlines() # readlines() crea en cada linea una lista de cadenas

In [24]:
# Hacemos un dataframe df_items
df_items = [eval(line.strip()) for line in data]
df_items = pd.DataFrame(df_items)

In [25]:
# Desanidar la columna 'items'
data_desanidada_items = df_items.explode('items')

In [26]:
data_desanidada_items = pd.json_normalize(data_desanidada_items["items"])

In [27]:
# Concatenamos df_items y data_desanidada_items
df_items = pd.concat([df_items, data_desanidada_items], axis=1)

In [28]:
# Pasamos a string la columna "user_id"
df_items['user_id'] = df_items['user_id'].astype(str)


In [29]:
# Borramos los valores vacios de la columna user_id
df_items = df_items.dropna(subset= "user_id")

In [30]:
# Una vez transformados los datos y columnas hacemos una exportacion en archivo csv
df_items.to_csv("df_items.csv",index=False)

In [31]:
df_items = df_items.dropna(subset= "user_id")


In [32]:
df_items

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",10,Counter-Strike,6.0,0.0
1,js41637,888.0,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...",20,Team Fortress Classic,0.0,0.0
2,evcentric,137.0,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...",30,Day of Defeat,7.0,0.0
3,Riot-Punch,328.0,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...",40,Deathmatch Classic,0.0,0.0
4,doctr,541.0,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...",50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...,...
5170010,nan,NaN,NaN,NaN,NaN,373330,All Is Dust,0.0,0.0
5170011,nan,NaN,NaN,NaN,NaN,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,nan,NaN,NaN,NaN,NaN,521570,You Have 10 Seconds 2,4.0,4.0
5170013,nan,NaN,NaN,NaN,NaN,519140,Minds Eyes,3.0,3.0


### `Hacemos un join con las 3 tablas`

In [33]:
# Hacemos un join con la columna "user_id"
df = pd.merge(df_users,df_items, on="user_id", how="inner")

In [34]:
# Renombramos el nombre "app_name" de la tabla df_steam_games por "item_name" para poder hacer un join
df_steam_games = df_steam_games.rename(columns={'app_name': 'item_name'})

In [35]:
# Hacemos un Join con las columnas "item_name"
df_games = pd.merge(df,df_steam_games, on="item_name", how="inner")

In [36]:
# Borramos las columnas que no vamos a utilizar

df_games = df_games.drop(columns= ["helpful","items_count","steam_id", "playtime_2weeks","title","id",'user_url_x',"reviews","funny","posted","last_edited",'user_url_y','items','item_id_y','url','reviews_url','specs', 'early_access',"publisher"])


###  `Análisis de sentimiento con NLP` 

In [37]:
# Importamos librerias
import nltk
from textblob import TextBlob

In [38]:
# Definir una función para el análisis de sentimiento
def analisis_sentimiento(comentario):
    blob = TextBlob(str(comentario))
    Polaridad = blob.sentiment.polarity

    if Polaridad > 0:
        return 2
    elif Polaridad < 0:
        return 0
    else:
        return 1

In [39]:
# Aplicar la función de análisis de sentimiento a la columna "review" del DataFrame
df_games['sentiment_analysis'] = df_games['review'].apply(lambda x: analisis_sentimiento(x))

In [40]:
# Reemplazar "review" con "sentiment_analysis"
df_games.drop(columns= "review")

,user_id,item_id_x,recommend,item_name,playtime_forever,genres,release_date,tags,price,developer,sentiment_analysis
0,76561197970982479,1250,True,Counter-Strike,6.0,[Action],2000.0,"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99,Valve,2
1,76561198060901683,313120,True,Counter-Strike,0.0,[Action],2000.0,"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99,Valve,2
2,76561198094455687,200210,False,Counter-Strike,337.0,[Action],2000.0,"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99,Valve,0
3,FireHalo,620,True,Counter-Strike,5.0,[Action],2000.0,"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99,Valve,2
4,xXGeronimo,295110,True,Counter-Strike,1.0,[Action],2000.0,"[Action, FPS, Multiplayer, Shooter, Classic, T...",9.99,Valve,2
...,...,...,...,...,...,...,...,...,...,...,...
16710,76561198155824473,220460,True,TowerFall Ascension,8.0,"[Action, Indie]",2014.0,"[Local Multiplayer, Indie, Local Co-Op, Action...",14.99,Matt Thorson,2
16711,Azzaaaa,222880,True,Blood of the Werewolf,0.0,"[Action, Indie]",2014.0,"[Platformer, Action, Indie, Werewolves, Female...",1.99,Scientifically Proven,1
16712,uncleshenny,730,True,The Last Federation,63.0,"[Indie, Simulation, Strategy]",2014.0,"[Strategy, Space, Simulation, Indie, Turn-Base...",19.99,"Arcen Games, LLC",0
16713,76561198161220449,4000,True,The Deed,1.0,"[Adventure, RPG]",2015.0,"[RPGMaker, Adventure, Crime, Atmospheric, Dete...",0.99,"Pilgrim Adventures,GrabTheGames Studios",2


In [5]:
# Una vez realizado todas las transformaciones y obtener una sola tabla procedemos a exportar el dataframe 
df_games.to_csv("df_games.csv",index=False)

NameError: name 'df_games' is not defined

###  `Funciones para el deployment de la API` 

In [122]:
import pandas as pd
df_games = pd.read_csv("df_games.csv")

####  Developer( desarrollador : str ):

Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora

In [129]:
def Developer(developer:str):

    df_developer = df_games[df_games["developer"] == developer]
    # Calcular el total de aplicaciones por año
    Cantidad_Items = df_developer.groupby('release_date')['item_id_x'].count().reset_index()


    # Los que tienen precio lo instanciamos como "No Free"
    df_developer.loc[df_developer["price"].apply(lambda x: isinstance(x, float)), "price"] = "No Free"
    
    # Agrupamos por año y precios "Free" y "No Free"
    Porcentaje = df_developer.groupby(["release_date", "price"])["price"].count().unstack()
    # Calcula el porcentaje de elementos "Free" por año
    
    if 'Free' in Porcentaje.columns:
        Porcentaje["Porcentaje_Free"] = (Porcentaje["Free"] / (Porcentaje["Free"])) * 100
        
    
    
    elif 'Free' in Porcentaje.columns and 'No Free' in Porcentaje.columns:
        Porcentaje["Porcentaje_Free"] = (Porcentaje["Free"] / (Porcentaje["Free"] + Porcentaje["No Free"])) * 100 

    Porcentaje = Porcentaje.fillna(0)
        
    Años = df_developer.groupby('release_date')["item_id_x"].count().reset_index()

    Porcentaje_Free = list(Porcentaje["Porcentaje_Free"].map("{:.2f}%".format))
    Años = list(Cantidad_Items["release_date"])
    Cantidad_Items = list(Cantidad_Items["item_id_x"])

    # Hacemos un zip con las tres listas 
    listas_unidas = list(zip(Años,Cantidad_Items, Porcentaje_Free))

    # Agregamos las claves para cada indice
    claves = ['Año', 'Cantidad', 'Porcentaje']
    diccionario = [dict(zip(claves, fila)) for fila in listas_unidas]

    return diccionario

In [131]:
Developer("Valve")

[{'Año': 1998.0, 'Cantidad': 33, 'Porcentaje': '0.00%'},
 {'Año': 1999.0, 'Cantidad': 31, 'Porcentaje': '0.00%'},
 {'Año': 2000.0, 'Cantidad': 74, 'Porcentaje': '0.00%'},
 {'Año': 2001.0, 'Cantidad': 19, 'Porcentaje': '0.00%'},
 {'Año': 2003.0, 'Cantidad': 26, 'Porcentaje': '0.00%'},
 {'Año': 2004.0, 'Cantidad': 310, 'Porcentaje': '0.00%'},
 {'Año': 2005.0, 'Cantidad': 86, 'Porcentaje': '100.00%'},
 {'Año': 2006.0, 'Cantidad': 137, 'Porcentaje': '0.00%'},
 {'Año': 2007.0, 'Cantidad': 148, 'Porcentaje': '0.00%'},
 {'Año': 2008.0, 'Cantidad': 33, 'Porcentaje': '0.00%'},
 {'Año': 2009.0, 'Cantidad': 165, 'Porcentaje': '0.00%'},
 {'Año': 2010.0, 'Cantidad': 26, 'Porcentaje': '0.00%'},
 {'Año': 2011.0, 'Cantidad': 90, 'Porcentaje': '0.00%'},
 {'Año': 2012.0, 'Cantidad': 240, 'Porcentaje': '0.00%'}]

#### UserForGenre( genero : str ):

devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

In [20]:
def UserForGenre(genero):

  # Hacemos un dataframe con el genero que le indicamos en la funcion
  df_genero = df_games[df_games["genres"]== genero]

  # Agrupamos por Usuario el dataframe df_genero
  df_agrupados = df_genero.groupby(["genres","user_id"])["playtime_forever"].sum().reset_index()

  # Usuario que acumulo mas horas jugadas para el genero dado
  if not df_agrupados.empty:
    Usuario_Horas_Jugadas = df_agrupados[df_agrupados["playtime_forever"]== df_agrupados["playtime_forever"].max()]["user_id"].values[0]
  else:
    Usuario_Horas_Jugadas = None
  # LISTA ACUMULADA POR AÑO
  # Hacemos un dataframe para el usuario que mas horas jugo con sus años ordenados
  df_Horas_Acumulado = df_genero.sort_values(by='release_date')
  df_Horas_Acumulado = df_genero[df_genero["user_id"] == Usuario_Horas_Jugadas]

  # Calculamos la horas y años de acumulacion y hacemos una lista
  Acumulacion_Horas = df_Horas_Acumulado.groupby('release_date')['playtime_forever'].sum().cumsum()
  Lista_Horas = Acumulacion_Horas.tolist()

  Acumulacion_Años = df_Horas_Acumulado.groupby('release_date')['playtime_forever'].count().reset_index()
  Lista_Años = Acumulacion_Años["release_date"].tolist()

  # Crear un diccionario que contenga las dos listas
  data = {
    'Años': Lista_Años,
    'Horas': Lista_Horas
    }
      
  


  return f"Usuario con más horas jugadas para {genero}: {data}"

In [21]:
UserForGenre("['Action', 'Indie']")

"Usuario con más horas jugadas para ['Action', 'Indie']: {'Años': [2013.0], 'Horas': [38740.0]}"

#### Userdata( User_id : str )

Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [22]:
def Userdata(User_id : str):
    
    # Agrupamos por usuario
    df_usuario = df_games[df_games["user_id"]== User_id]

    # Convertir los valores de la columna a números (omitir los strings)
    df_usuario['price'] = pd.to_numeric(df_usuario['price'], errors='coerce')
    # Sumamos el dinero total gastado del usuario
    df_dinero_gastado = df_usuario['price'].sum()

    # Obtenemos el porcentaje de recomendacion
    Porcentaje_Recomendacion = df_usuario["recommend"].mean() * 100
    Porcentaje_Recomendacion

    Cantidad_items = df_usuario["item_id_x"].count()
    
    #{"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}
    diccionario = f"Usuario X: {User_id}, Dinero gastado: {df_dinero_gastado} USD, % de recomendación: {Porcentaje_Recomendacion}%, Cantidad de items: {Cantidad_items} "
            
    
    return diccionario




In [23]:
Userdata("DrMolo")

C:\Users\Giuliano\AppData\Local\Temp\ipykernel_4116\3147296809.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_usuario['price'] = pd.to_numeric(df_usuario['price'], errors='coerce')


'Usuario X: DrMolo, Dinero gastado: 79.98 USD, % de recomendación: 100.0%, Cantidad de items: 4 '

#### Best_developer_year( año : int ): 

Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [50]:
def Best_developer_year(año:int):

    
    df_desarrolador = df_games[df_games["release_date"] == año]
    # Agrupamos por desarrolador y sumamos las remcomendaciones 
    df_agrupados = df_desarrolador.groupby(["developer"])["recommend"].sum().reset_index()
    # Ordenamos los top 3 desarroladores mas recomendados
    Juego_mas_recomendado = df_agrupados.sort_values(by='recommend', ascending=False).head(3)

    data = {
            'Puesto 1': Juego_mas_recomendado.iloc[0]["developer"],
            'Puesto 2': Juego_mas_recomendado.iloc[1]["developer"],
            'Puesto 3': Juego_mas_recomendado.iloc[2]["developer"]
        }

    return data

In [51]:
Best_developer_year(2001)

{'Puesto 1': 'Gearbox Software',
 'Puesto 2': 'Valve',
 'Puesto 3': 'MicroProse Software, Inc'}

#### Developer_reviews_analysis( desarrolladora : str ):

Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [52]:
def developer_reviews_analysis( desarrolladora : str ):

    df_desarrolador = df_games[df_games["developer"] == desarrolladora]

    Cantidad_Registros = df_desarrolador["sentiment_analysis"].value_counts().reset_index()

    positivos = f"Positivos = {Cantidad_Registros.iloc[0][1]}"
    negativos =  f"Negativos = {Cantidad_Registros.iloc[2][1]}"
    usuario = desarrolladora
    data = {
                usuario:[positivos, negativos]
                }

    return data
    


In [53]:
developer_reviews_analysis("Facepunch Studios")

{'Facepunch Studios': ['Positivos = 151', 'Negativos = 64']}

###  `Modelo de aprendizaje automático` 

In [2]:
import pandas as pd
df_games = pd.read_csv("df_games.csv")

In [31]:
def Recomendacion_juego(item_name:str):
    
    from sklearn.metrics.pairwise import cosine_similarity

    # Crear una matriz de usuario-item
    user_item_matrix = pd.pivot_table(df_games, values='playtime_forever', index='user_id', columns='item_name', fill_value=0)

    # Calcular la similitud de coseno entre juegos
    game_similarity = cosine_similarity(user_item_matrix.T)

    # Encontrar el índice del juego en la matriz
    game_index = user_item_matrix.columns.get_loc(item_name)
    

    # Calcular la similitud de coseno entre el juego deseado y otros juegos
    game_similarities = game_similarity[game_index] 

    # Crear un DataFrame con juegos similares y sus similitudes
    similar_games = pd.DataFrame({
        'Game': user_item_matrix.columns,
        'Similarity': game_similarities
    })

    # Ordenar los juegos por similitud en orden descendente
    top_n_recommendations = similar_games.sort_values(by='Similarity', ascending=False)

    # Imprimir los 5 juegos mas recomendados
    
    top_n_recommendations = list(top_n_recommendations["Game"][1:6])
    
    return top_n_recommendations

In [34]:
Recomendacion_juego('Company of Heroes 2')

['Ride! Carnival Tycoon',
 'Reus',
 'Reveal The Deep',
 'Revenge of the Titans',
 'Revolution Ace']